In [5]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import numpy as np
import pandas as pd
import xgboost as xgb
import pickle
import pandas as pd
import xgboost as xgb
# from sklearn.externals import joblib

In [3]:
# 전역 dict 작성
int2name = {0 : 'train_', 1 : 'test1_', 2 : 'test2_'}

In [4]:
# preprocess에서 데이터 호출
def load_preprocess_data(is_test=0):
    name = int2name[is_test]
    path = '../preprocess/'
    # 데이터 불러오기
    data_for_surv = pd.read_csv(path + name + 'preprocess_1.csv')
    data_for_spent = pd.read_csv(path + name + 'preprocess_2.csv')
    return data_for_surv, data_for_spent

In [6]:
data_for_surv, data_for_spent = load_preprocess_data(0)

In [8]:
data_for_surv.head()

,acc_id,playtime_count,solo_exp_count,quest_exp_count,fishing_count,game_money_change_count,login_clt,deff_level,level_max,day_count,...,temp_cnt_3_y,combat_play_time_3,pledge_rank_3,combat_char_cnt_4,temp_cnt_4_y,etc_cnt_4_y,combat_play_time_4,pledge_rank_4,amount_spent_1,amount_spent_4
0,2,28,0,0,0,28,0,0,0,28,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,5,10,10,7,3,10,1,0,15,10,...,0.296649,0.593982,1049.0,1.046590,1.384363,0.417618,3.661882,66814.0,0.000000,0.000000
2,8,28,28,25,11,28,0,6,16,28,...,20.963206,33.515247,35879.0,12.270369,21.556504,6.243030,29.782361,22331.0,1.056123,0.348521
3,17,28,27,25,15,28,0,1,16,28,...,8.009527,14.629460,2673.0,5.738202,10.778252,2.526941,15.140507,2673.0,0.000000,0.000000
4,20,27,26,24,9,27,0,0,17,27,...,5.339685,14.720272,1080.0,7.903561,7.317346,11.672059,18.448062,2500.0,0.000000,0.000000


In [9]:
data_for_spent.head()

,acc_id,playtime,quest_exp,game_money_change,playtime_count,npc_kill_count,solo_exp_count,quest_exp_count,game_money_change_count,day_attack,...,combat_play_time_3,play_char_cnt_4,pledge_rank_4,amount_spent_1,amount_spent_3,amount_spent_4,week1_log,week2_log,week3_log,week4_log
0,2,91.533663,0.000000,-0.008746,28,0,0,0,28,0,...,0.000000,0.000000,0,0.000000,0.000000,0.000000,1.0,1.0,1.000000,1.000000
1,5,6.057616,0.111055,0.145146,10,10,10,7,10,0,...,0.593982,4.005914,66814,0.000000,0.000000,0.000000,0.0,0.0,0.142857,0.142857
2,8,9.142599,0.067673,0.002802,28,27,28,25,28,1,...,33.515247,40.961377,22331,1.056123,0.000000,0.348521,1.0,1.0,1.000000,1.000000
3,17,25.213538,0.079819,-0.024625,28,28,27,25,28,0,...,14.629460,16.276283,2673,0.000000,0.000000,0.000000,1.0,1.0,1.000000,1.000000
4,20,32.357218,0.025051,-0.391668,27,27,26,24,27,0,...,14.720272,15.734943,2500,0.000000,0.826123,0.000000,1.0,1.0,1.000000,1.000000


In [11]:
def load_model():
    path = '../model/'
    # 모델 불러오기
    survival_time_model = pickle.load(open(path + 'survival_time_model.pkl', 'rb'))
    amount_spent_model = pickle.load(open(path + 'amount_spent_model.pkl', 'rb'))
    return survival_time_model, amount_spent_model

In [12]:
survival_time_model, amount_spent_model = load_model()

In [13]:
survival_time_model

In [14]:
amount_spent_model

In [15]:
survival_pred = survival_time_model.predict(
    xgb.DMatrix(
        data_for_surv[data_for_surv.columns[1:]].values
    )
)

In [17]:
survival_pred.shape

(40000,)

In [27]:
y_pred = pd.DataFrame(data=np.vstack((data_for_surv['acc_id'].values, survival_pred)).T, columns=['acc_id', 'survival_time'])
y_pred.head()

,acc_id,survival_time
0,2.0,97.005318
1,5.0,47.859837
2,8.0,105.224709
3,17.0,105.818367
4,20.0,73.984703


In [31]:
y_pred['survival_time'] = round(y_pred['survival_time']).values

In [34]:
y_pred['survival_time'] = (y_pred['survival_time'] - 32).values

In [36]:
y_pred['survival_time'] = np.where(y_pred['survival_time'] <= 0, 1,
    np.where(y_pred['survival_time'] >= 64, 64, y_pred['survival_time']))

In [37]:
y_pred.head()

,acc_id,survival_time
0,2.0,64.0
1,5.0,16.0
2,8.0,64.0
3,17.0,64.0
4,20.0,42.0


In [38]:
spent_pred = amount_spent_model.predict(
    xgb.DMatrix(
        data_for_spent[data_for_spent.columns[1:]].values
    )
)

In [40]:
y_pred['amount_spent'] = spent_pred

In [41]:
y_pred['amount_spent'] = np.where(y_pred['amount_spent'] < 0,
                                  0, y_pred['amount_spent'])

In [42]:
y_pred.head()

,acc_id,survival_time,amount_spent
0,2.0,64.0,0.066259
1,5.0,16.0,0.211025
2,8.0,64.0,0.014291
3,17.0,64.0,0.063998
4,20.0,42.0,0.153062
